# Imports

In [ ]:
import os
import time
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats
from scipy.io.arff import loadarff
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# Helper functions

In [ ]:
# função para criar os diretórios
def creat_dir(model_name):
    if not 'log_metrics' in os.listdir():
        os.mkdir('log_metrics')
    if not 'imgs' in os.listdir():
        os.mkdir('imgs')
    if not model_name in os.listdir('./imgs'):
        os.mkdir('./imgs/'+model_name)

In [ ]:
# função para adicionar as métricas de um fold em um dicionário
def dic_par_metrics(y_test, y_onehot_test, y_pred, y_proba, grid_search_cv):
    accuracy = metrics.accuracy_score(y_test, y_pred)
    balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
    precision_micro = metrics.precision_score(y_test, y_pred, average='micro')
    precision_macro = metrics.precision_score(y_test, y_pred, average='macro')
    precision_weighted = metrics.precision_score(y_test, y_pred, average='weighted')
    recall_micro = metrics.recall_score(y_test, y_pred, average='micro')
    recall_macro = metrics.recall_score(y_test, y_pred, average='macro')
    recall_weighted = metrics.recall_score(y_test, y_pred, average='weighted')
    f1_micro = metrics.f1_score(y_test, y_pred, average='micro')
    f1_macro = metrics.f1_score(y_test, y_pred, average='macro')
    f1_weighted = metrics.f1_score(y_test, y_pred, average='weighted')
    roc_auc_ovr_micro = metrics.roc_auc_score(y_onehot_test, y_proba, multi_class='ovr', average='micro')
    roc_auc_ovo_micro = metrics.roc_auc_score(y_onehot_test, y_proba, multi_class='ovo', average='micro')
    roc_auc_ovr_macro = metrics.roc_auc_score(y_onehot_test, y_proba, multi_class='ovr', average='macro')
    roc_auc_ovo_macro = metrics.roc_auc_score(y_onehot_test, y_proba, multi_class='ovo', average='macro')
    roc_auc_ovr_weighted = metrics.roc_auc_score(y_onehot_test, y_proba, multi_class='ovr', average='weighted')
    roc_auc_ovo_weighted = metrics.roc_auc_score(y_onehot_test, y_proba, multi_class='ovo', average='weighted')
    classification_report = metrics.classification_report(y_test, y_pred, output_dict=True)
    dicMetricas = {
        "parameters": grid_search_cv.best_params_,
        "metrics":{
            "accuracy": accuracy,
            "balanced_accuracy": balanced_accuracy,
            "precision_micro": precision_micro,
            "precision_macro": precision_macro,
            "precision_weighted": precision_weighted,
            "recall_micro": recall_micro,
            "recall_macro": recall_macro,
            "recall_weighted": recall_weighted,
            "f1_micro": f1_micro,
            "f1_macro": f1_macro,
            "f1_weighted": f1_weighted,
            "roc_auc_ovr_micro": roc_auc_ovr_micro,
            "roc_auc_ovo_micro": roc_auc_ovo_micro,
            "roc_auc_ovr_macro": roc_auc_ovr_macro,
            "roc_auc_ovo_macro": roc_auc_ovo_macro,
            "roc_auc_ovr_weighted": roc_auc_ovr_weighted,
            "roc_auc_ovo_weighted": roc_auc_ovo_weighted
        },
        "classification_report": classification_report
    }
    return dicMetricas

In [ ]:
# função para calcular a média e o desvio padrão das métricas de cada fold, além de plotar os boxplots
def calc_mean_std(dic_json, model_name):
    key = list(dic_json.keys())[0]
    dic_mean = {}
    dic_std = {}
    for j in dic_json[key]['metrics'].keys():
        metrics_list = []
        for i in dic_json.keys():
            metrics_list.append(dic_json[i]['metrics'][j])
        mean = np.mean(metrics_list)
        std = np.std(metrics_list)
        dic_mean.update({j: mean})
        dic_std.update({j: std})
        plt.close('all')
        plt.boxplot(metrics_list, labels=[model_name])
        plt.ylabel(j)
        plt.savefig('./imgs/'+model_name+'/boxplot_'+j+'.png')
        plt.close('all')
    dic_json.update({"mean": dic_mean})
    dic_json.update({"std": dic_std})
    return dic_json

In [ ]:
def plot_matriz_confusao_one_vs_one(y_test, y_pred, model_name, fold_i, flag_normalizado="true"):
    """Plota a matriz de confusao comparando cada classe entre si, mostrando as predicoes contra as
    classes verdadeiras.

    Args:
        y_test: serie com os valores verdadeiros
        y_pred: serie com os valores preditos
        flag_normalizado (optional): flag indicando se os valores da matriz de confusao devem ser
        normalizados. Se devem ser normalizados pel. Defaults to None.
    """
    class_names = np.unique(y_pred)

    fig, ax = plt.subplots(figsize=(12, 10))
    cm_plot = metrics.ConfusionMatrixDisplay.from_predictions(
        y_test, y_pred, cmap="YlOrRd", normalize=flag_normalizado, ax=ax)
    ax.set_xticklabels(class_names, rotation=45, ha="right")
    ax.set_title(f"Matriz de Confusao - Modelo {model_name} - Fold {fold_i}")

    cm_plot.figure_.savefig(
        f"imgs/{model_name}/cm_normalizado_{flag_normalizado}_{model_name}_fold_{fold_i}.png", 
        dpi=300
    )
    plt.close()
    
    return cm_plot

In [ ]:
def inicializa_roc_one_class_vs_rest_kfold(y_onehot_test, y_pred_score, i, class_id, ax, tprs, aucs, mean_fpr):
    viz = metrics.RocCurveDisplay.from_predictions(
        y_onehot_test[:, class_id],
        y_pred_score[:, class_id],
        name=f"ROC OvR Class_{class_id+1} fold {i}",
        alpha=0.3,
        lw=1,
        ax=ax,
        plot_chance_level=(i==9),
    )

    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

In [ ]:
def gera_roc_one_class_vs_rest_kfold(tprs, aucs, mean_fpr, plot, model_name, class_id):
    fig = plot[0]
    ax = plot[1]
    
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = metrics.auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )
    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title=f"Curva ROC media com desvio padrao\nROC OvR Class_{class_id+1}",
    )
    ax.axis("square")
    ax.legend(loc="lower right")

    fig.savefig(f"imgs/{model_name}/roc_{model_name}_class_{class_id+1}.png", dpi=300)
    plt.close()

In [ ]:
def inicializa_roc_micro_average_kfold(y_onehot_test, y_pred_score, i, ax, tprs, aucs, mean_fpr):
    viz = metrics.RocCurveDisplay.from_predictions(
        y_onehot_test.ravel(),
        y_pred_score.ravel(),
        name=f"micro-average OvR fold {i}",
        alpha=0.3,
        lw=1,
        ax=ax,
        plot_chance_level=(i==9)
    )

    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

In [ ]:
def gera_roc_micro_average_kfold(tprs, aucs, mean_fpr, fig, ax, model_name):
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = metrics.auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )
    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title=f"Curva ROC media com desvio padrao\nROC micro-average",
    )
    ax.axis("square")
    ax.legend(loc="lower right")

    fig.savefig(f"imgs/{model_name}/roc_micro_average_{model_name}.png", dpi=300)
    plt.close()

In [ ]:
def stratified_k_fold_grid_search_cv(
        model, params:dict, X, y, model_name, flag_smote=False, flag_variance_threshold=False,
        flag_pca=False
):
    """Realiza o treino e validacao do modelo utilizando StratifiedKFold com GridSearchCV para
    busca dos melhores hiperparametros (tendo assim um nested cross-validation).

    Args:
        model: modelo que se deseja treinar e validar
        params (dict): dicionario com os parametros e seus respectivos valores possiveis para o 
        grid search
        X: features do modelo
        y: coluna de target do modelo
    """
    startTime = time.time()
    # criando os diretórios necessários para salvar as métricas e os plots
    creat_dir(model_name)

    print(f"Modelo: {model_name}")
    dic_json = {}

    n_splits_k_fold = 10
    n_splits_grid_search = 5

    matriz_tprs = [[] for i in range(9)]
    matriz_aucs = [[] for i in range(9)]
    mean_fpr = np.linspace(0, 1, 100)
    lista_plots = [plt.subplots(figsize=(6, 6)) for i in range(9)]

    tprs_micro_average = []
    aucs_micro_average = []
    fig_micro_average, ax_micro_average = plt.subplots(figsize=(6, 6))

    skf = StratifiedKFold(n_splits=n_splits_k_fold, random_state=4, shuffle=True)
    for i, (train_index, test_index) in enumerate(skf.split(X, y)):
        print(f"## INICIO FOLD {i} ##")
        # separa os dados de treino e teste
        X_train = X.iloc[train_index]
        X_test = X.iloc[test_index]
        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]
        # realiza a tecnica SMOTE no dados de treino para tratamento do desbalanceamento
        if flag_smote:
            smote = SMOTE(random_state=4)
            X_train, y_train = smote.fit_resample(X_train, y_train)
        # realiza a tecnica de selecao de atributos utilizando Variance Threshold
        if flag_variance_threshold:
            filter_variance = VarianceThreshold(0.8)
            X_train = filter_variance.fit_transform(X_train)
            X_test = filter_variance.transform(X_test)
        # normaliza os dados
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        # realiza a tecnica de extracao de caracteristicas e reducao de dimensionalidade PCA
        if flag_pca:
            # mantem apenas os componentes principais que contem 90% da informacao dos dados 
            # originais
            pca = PCA(n_components=0.9, svd_solver="full")
            X_train = pca.fit_transform(X_train)
            X_test = pca.transform(X_test)
        # binariza a coluna target para uso no plot roc e metrica roc_auc_score
        label_binarizer = LabelBinarizer().fit(y_train)
        y_onehot_test = label_binarizer.transform(y_test)
        # prepara o y_train
        y_train = y_train.values.ravel()
        # inicializa e roda o grid search
        grid_search_cv = GridSearchCV(
            estimator=model, param_grid=params, scoring="f1_weighted", cv=n_splits_grid_search, 
            verbose=3)
        grid_search_cv.fit(X_train, y_train)
        # com os melhores parametros encontrados, realiza a predicao no fold de teste e calcula a 
        # metrica de avaliacao
        y_pred = grid_search_cv.predict(X_test)
        # verifica se o modelo em execucao eh o perceptron, ja que o mesmo nao possui a funcao 
        # predict_proba
        if model_name != 'Perceptron':
            y_proba = grid_search_cv.predict_proba(X_test)
        else:
            y_proba = grid_search_cv.decision_function(X_test)
        # armazenando as metricas e os parametros em um dicionario
        dic_fold = dic_par_metrics(y_test, y_onehot_test, y_pred, y_proba, grid_search_cv)
        dic_json.update({"fold "+str(i): dic_fold})
        # gera a matriz de confusao com dados normalizados e nao normalizados e salva ambas versoes
        plot_matriz_confusao_one_vs_one(y_test, y_pred, model_name, i, flag_normalizado="true")
        plot_matriz_confusao_one_vs_one(y_test, y_pred, model_name, i, flag_normalizado=None)
        # inicializa a curva roc para cada classe neste fold
        for class_id, plot, tprs, aucs in zip(range(9), lista_plots, matriz_tprs, matriz_aucs):
            inicializa_roc_one_class_vs_rest_kfold(
                y_onehot_test, y_proba, i, class_id, plot[1], tprs, aucs, mean_fpr
            )
        # inicializa a curva roc micro average neste fold
        inicializa_roc_micro_average_kfold(
            y_onehot_test, y_proba, i, ax_micro_average, tprs_micro_average, 
            aucs_micro_average, mean_fpr
        )
        print(f"## FINAL FOLD {i} ##\n", dic_fold)

    endTime = time.time()
    # inserindo a média dos folds no dicionário
    dic_json = calc_mean_std(dic_json, model_name)
    # inserindo o tempo de execução
    dic_json.update({"time": endTime-startTime})
    # salvando o dicionário no formato json
    objOpen = open(f'./log_metrics/{model_name}.json', 'w')
    objOpen.write(json.dumps(dic_json, indent=4))
    objOpen.close()

    # gera o plot da curva roc para cada classe, juntando todos os folds e realizando uma media
    for class_id, plot, tprs, aucs in zip(range(9), lista_plots, matriz_tprs, matriz_aucs):
        gera_roc_one_class_vs_rest_kfold(tprs, aucs, mean_fpr, plot, model_name, class_id)
    # gera o plot da curva roc micro average, juntando todos os folds e realizando uma media 
    gera_roc_micro_average_kfold(
        tprs_micro_average, aucs_micro_average, mean_fpr, fig_micro_average, ax_micro_average, 
        model_name
    )

# Carrega dados

In [ ]:
raw_data = loadarff('data/dataset.arff')
df_data = pd.DataFrame(raw_data[0])
df_data.head()

In [ ]:
# decodifica string de target
df_data["target"] = df_data["target"].str.decode("utf-8")

In [ ]:
df_data.sample(5)

In [ ]:
# verificando o número de linhas e colunas do dataset
print("Shape dataset:", df_data.shape)

In [ ]:
# verificando o menor valor das colunas de características
print("Valor mínimo das features:", df_data.iloc[:,1:-1].min().min())

In [ ]:
# verificando o maior valor das colunas de características
print("Valor máximo das features:", df_data.iloc[:,1:-1].max().max())

In [ ]:
# verificando o número de instâncias para cada classe
print('Número de Instâncias por Classe:\n', df_data.groupby('target').count()['id'])

In [ ]:
# verificando o percentual de instâncias para cada classe
print('Percentual de Instâncias por Classe:\n', (df_data.groupby('target').count()['id']/df_data.shape[0])*100)

In [ ]:
# verificando se existem valores NaN
print('\nNúmero de Valores NaN:', df_data.isnull().sum().sum())

In [ ]:
df_data.describe()

# Modelos

In [ ]:
X = df_data.drop(columns=["id", "target"])
y = df_data[["target"]]

## KNN

### Sem pré-processamento

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "n_neighbors": [5, 10, 15]
}
# define o modelo
model = KNeighborsClassifier(n_jobs=1)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "KNN")

### SMOTE

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "n_neighbors": [5, 10, 15]
}
# define o modelo
model = KNeighborsClassifier(n_jobs=1)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "KNN_smote", flag_smote=True)

### Variance Threshold

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "n_neighbors": [5, 10, 15]
}
# define o modelo
model = KNeighborsClassifier(n_jobs=1)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(
    model, params, X, y, "KNN_variance_threshold", flag_variance_threshold=True
)

### PCA

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "n_neighbors": [5, 10, 15]
}
# define o modelo
model = KNeighborsClassifier(n_jobs=1)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(
    model, params, X, y, "KNN_pca", flag_pca=True
)

## DecisionTree

### Sem Pré-Processamento

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "criterion": ["gini", "entropy", "log_loss"],
    "class_weight": ["balanced", None],
    "max_depth": [None, 50, 100, 200, 500, 1000]
}
# define o modelo
model = DecisionTreeClassifier(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "DecisionTree")

### SMOTE

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "criterion": ["gini", "entropy", "log_loss"],
    "class_weight": ["balanced", None],
    "max_depth": [None, 50, 100, 200, 500, 1000]
}
# define o modelo
model = DecisionTreeClassifier(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "DecisionTree_smote", flag_smote=True)

### Variance Threshold

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "criterion": ["gini", "entropy", "log_loss"],
    "class_weight": ["balanced", None],
    "max_depth": [None, 50, 100, 200, 500, 1000]
}
# define o modelo
model = DecisionTreeClassifier(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(
    model, params, X, y, "DecisionTree_variance_threshold", flag_variance_threshold=True
)

### PCA

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "criterion": ["gini", "entropy", "log_loss"],
    "class_weight": ["balanced", None],
    "max_depth": [None, 50, 100, 200, 500, 1000]
}
# define o modelo
model = DecisionTreeClassifier(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(
    model, params, X, y, "DecisionTree_pca", flag_pca=True
)

### Avaliação efeito PCA

#### Sem PCA

In [ ]:
X_train_dt_sem_pca, X_test_dt_sem_pca, y_train_dt_sem_pca, y_test_dt_sem_pca = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=4)
# define o modelo
model_dt_sem_pca = DecisionTreeClassifier(random_state=4)
# normaliza os dados
scaler_dt_sem_pca = StandardScaler()
X_train_dt_sem_pca = scaler_dt_sem_pca.fit_transform(X_train_dt_sem_pca)
X_test_dt_sem_pca = scaler_dt_sem_pca.transform(X_test_dt_sem_pca)
# inicializa e roda o grid search
model_dt_sem_pca.fit(X_train_dt_sem_pca, y_train_dt_sem_pca)
# com os melhores parametros encontrados, realiza a predicao no fold de teste e calcula a 
# metrica de avaliacao
y_pred_dt_sem_pca = model_dt_sem_pca.predict(X_test_dt_sem_pca)

In [ ]:
model_dt_sem_pca.get_n_leaves()

In [ ]:
model_dt_sem_pca.get_depth()

#### Com PCA

In [ ]:
X_train_dt_com_pca, X_test_dt_com_pca, y_train_dt_com_pca, y_test_dt_com_pca = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=4)
# define o modelo
model_dt_com_pca = DecisionTreeClassifier(random_state=4)
# normaliza os dados
scaler_dt_com_pca = StandardScaler()
X_train_dt_com_pca = scaler_dt_com_pca.fit_transform(X_train_dt_com_pca)
X_test_dt_com_pca = scaler_dt_com_pca.transform(X_test_dt_com_pca)
# realiza a tecnica de extracao de caracteristicas e reducao de dimensionalidade PCA
# mantem apenas os componentes principais que contem 90% da informacao dos dados 
# originais
pca_dt_com_pca = PCA(n_components=0.9, svd_solver="full")
X_train_dt_com_pca = pca_dt_com_pca.fit_transform(X_train_dt_com_pca)
X_test_dt_com_pca = pca_dt_com_pca.transform(X_test_dt_com_pca)
# inicializa e roda o grid search
model_dt_com_pca.fit(X_train_dt_com_pca, y_train_dt_com_pca)
# com os melhores parametros encontrados, realiza a predicao no fold de teste e calcula a 
# metrica de avaliacao
y_pred_dt_com_pca = model.predict(X_test_dt_com_pca)

In [ ]:
model_dt_com_pca.get_n_leaves()

In [ ]:
model_dt_com_pca.get_depth()

## Random Forest

### Sem Pré-Processamento

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    #"n_estimators": [100, 300, 500],
    "n_estimators": [300, 500],
    #"max_depth": [None, 400, 800],
    "class_weight": ["balanced_subsample", None]
}

# define o modelo
model = RandomForestClassifier(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "RandomForest")

### SMOTE

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    #"n_estimators": [100, 300, 500],
    "n_estimators": [300, 500],
    #"max_depth": [None, 400, 800],
    "class_weight": ["balanced_subsample", None]
}

# define o modelo
model = RandomForestClassifier(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "RandomForest_smote", flag_smote=True)

### Variance Threshold

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    #"n_estimators": [100, 300, 500],
    "n_estimators": [300, 500],
    #"max_depth": [None, 400, 800],
    "class_weight": ["balanced_subsample", None]
}

# define o modelo
model = RandomForestClassifier(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(
    model, params, X, y, "RandomForest_variance_threshold", flag_variance_threshold=True
)

### PCA

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    #"n_estimators": [100, 300, 500],
    "n_estimators": [300, 500],
    #"max_depth": [None, 400, 800],
    "class_weight": ["balanced_subsample", None]
}

# define o modelo
model = RandomForestClassifier(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "RandomForest_pca", flag_pca=True)

## Naive Bayes (Gaussian)

### Sem Pré-Processamento

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
}

# define o modelo
model = GaussianNB()
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "NaiveBayes")

### SMOTE

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
}

# define o modelo
model = GaussianNB()
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "NaiveBayes_smote", flag_smote=True)

### Variance Threshold

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
}

# define o modelo
model = GaussianNB()
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(
    model, params, X, y, "NaiveBayes_variance_threshold", flag_variance_threshold=True)

### PCA

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
}

# define o modelo
model = GaussianNB()
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "NaiveBayes_pca", flag_pca=True)

## Logistic Regression

### Sem Pré-Processamento

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "penalty": [None, "l2"],
    "class_weight": ["balanced", None],
    "max_iter": [100, 300, 500, 700, 1000]
}

# define o modelo
model = LogisticRegression(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "LogisticRegression")

### SMOTE

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "penalty": [None, "l2"],
    "class_weight": ["balanced", None],
    "max_iter": [100, 300, 500, 700, 1000]
}

# define o modelo
model = LogisticRegression(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "LogisticRegression_smote", flag_smote=True)

### Variance Threshold

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "penalty": [None, "l2"],
    "class_weight": ["balanced", None],
    "max_iter": [100, 300, 500, 700, 1000]
}

# define o modelo
model = LogisticRegression(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(
    model, params, X, y, "LogisticRegression_variance_threshold", flag_variance_threshold=True)

### PCA

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "penalty": [None, "l2"],
    "class_weight": ["balanced", None],
    "max_iter": [100, 300, 500, 700, 1000]
}

# define o modelo
model = LogisticRegression(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "LogisticRegression_pca", flag_pca=True)

## Perceptron

### Sem Pré-Processamento

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "penalty": [None, "l2"],
    "class_weight": ["balanced", None],
    "max_iter": [100, 300, 500, 700, 1000]
}

# define o modelo
model = Perceptron(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "Perceptron")

### SMOTE

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "penalty": [None, "l2"],
    "class_weight": ["balanced", None],
    "max_iter": [100, 300, 500, 700, 1000]
}

# define o modelo
model = Perceptron(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "Perceptron_smote", flag_smote=True)

### Variance Threshold

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "penalty": [None, "l2"],
    "class_weight": ["balanced", None],
    "max_iter": [100, 300, 500, 700, 1000]
}

# define o modelo
model = Perceptron(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(
    model, params, X, y, "Perceptron_variance_threshold", flag_variance_threshold=True)

### PCA

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "penalty": [None, "l2"],
    "class_weight": ["balanced", None],
    "max_iter": [100, 300, 500, 700, 1000]
}

# define o modelo
model = Perceptron(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "Perceptron_pca", flag_pca=True)

## MLP

### Sem Pré-Processamento

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "learning_rate_init": [0.001, 0.01],
    # "max_iter": [200, 500, 700],
    "max_iter": [200, 500],
    # "hidden_layer_sizes": [20, 50, 100]
    "hidden_layer_sizes": [50, 100]
}

# define o modelo
model = MLPClassifier(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "MLPClassifier")

### SMOTE

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "learning_rate_init": [0.001, 0.01],
    # "max_iter": [200, 500, 700],
    "max_iter": [200, 500],
    # "hidden_layer_sizes": [20, 50, 100]
    "hidden_layer_sizes": [50, 100]
}

# define o modelo
model = MLPClassifier(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "MLPClassifier_smote", flag_smote=True)

### Variance Threshold

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "learning_rate_init": [0.001, 0.01],
    # "max_iter": [200, 500, 700],
    "max_iter": [200, 500],
    # "hidden_layer_sizes": [20, 50, 100]
    "hidden_layer_sizes": [50, 100]
}

# define o modelo
model = MLPClassifier(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(
    model, params, X, y, "MLPClassifier_variance_threshold", flag_variance_threshold=True)

### PCA

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "learning_rate_init": [0.001, 0.01],
    # "max_iter": [200, 500, 700],
    "max_iter": [200, 500],
    # "hidden_layer_sizes": [20, 50, 100]
    "hidden_layer_sizes": [50, 100]
}

# define o modelo
model = MLPClassifier(random_state=4)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "MLPClassifier_pca", flag_pca=True)

## SVM

### Sem Pré-Processamento

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "C": [1],
    "kernel": ["rbf", "linear"]
}

# define o modelo
model = SVC(random_state=4, probability=True)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "SVM")

### PCA

In [ ]:
# define os parametros e seus respectivos valores a serem testados no grid search
params = {
    "C": [1],
    "kernel": ["rbf", "linear"]
}

# define o modelo
model = SVC(random_state=4, probability=True)
# chama a funcao que roda o stratified k fold e valida o modelo realizando a busca por hiper 
# parametros com grid search cv, fazendo assim um nested cross-validation
stratified_k_fold_grid_search_cv(model, params, X, y, "SVM_pca", flag_pca=True)

# Teste de Hipótese

Vamos avaliar se a média dos resultados obtidos com o modelo Random Forest é maior que a média dos resultados obtidos com o modelo MLP. Iremos realizar um teste de hipótese para checar se, estatisticamente, podemos constatar essa afirmação.

In [ ]:
with open("log_metrics/RandomForest.json") as json_file:
    json_rf = json.load(json_file)

lista_resultados_rf = []
for key in list(json_rf.keys()):
    if (key != "mean") & (key != "std") & (key != "time"):
        lista_resultados_rf.append(json_rf[key]["metrics"]["f1_weighted"])

with open("log_metrics/MLPClassifier.json") as json_file:
    json_mlp = json.load(json_file)

lista_resultados_mlp = []
for key in list(json_mlp.keys()):
    if (key != "mean") & (key != "std") & (key != "time"):
        lista_resultados_mlp.append(json_mlp[key]["metrics"]["f1_weighted"])

df_resultados = pd.DataFrame({"rf": lista_resultados_rf, "mlp": lista_resultados_mlp})

In [ ]:
fig, ax = plt.subplots(1, 2)
sns.set(rc={'figure.figsize':(10,5)})

sns.histplot(df_resultados, x="rf", ax=ax[0]).set_title("Distribuicao Random Forest")
sns.histplot(df_resultados, x="mlp", ax=ax[1]).set_title("Distribuicao MLP");

In [ ]:
print(
    f"Teste normalidade random forest: {stats.shapiro(df_resultados['rf'])}\n"
    f"Teste normalidade mlp: {stats.shapiro(df_resultados['mlp'])}"
)

In [ ]:
stats.mannwhitneyu(df_resultados["rf"], df_resultados["mlp"], alternative="greater")

In [ ]:
print(
    f"Variancia random forest: {np.var(df_resultados['rf'])}\n"
    f"Variancia mlp: {np.var(df_resultados['mlp'])}"
)

In [ ]:
stats.ttest_ind(a=df_resultados["rf"], b=df_resultados["mlp"], equal_var=True, alternative="greater")

# Cálculo da média das métricas de performance por classe

In [ ]:
model_name = "MLPClassifier_smote"
with open(f"log_metrics/{}.json") as json_file:
    data = json.load(json_file)
    precision_dict = {}
    precision_mean_dict = {}
    precision_std_dict = {}
    recall_dict = {}
    recall_mean_dict = {}
    recall_std_dict = {}
    f1_score_dict = {}
    f1_score_mean_dict = {}
    f1_score_std_dict = {}

    for fold in data:
        if "fold " in fold:
            for class_number in data[fold]["classification_report"]:
                if "Class_" in class_number:
                    if class_number not in precision_dict:
                        precision_dict[class_number] = []
                    if class_number not in recall_dict:
                        recall_dict[class_number] = []
                    if class_number not in f1_score_dict:
                        f1_score_dict[class_number] = []
                    for class_metric in data[fold]["classification_report"][class_number]:
                            if class_metric == "precision":
                                precision_dict[class_number].append(
                                    data[fold]["classification_report"][class_number][class_metric]
                                )
                            if class_metric == "recall":
                                recall_dict[class_number].append(
                                    data[fold]["classification_report"][class_number][class_metric]
                                )
                            if class_metric == "f1-score":
                                f1_score_dict[class_number].append(
                                    data[fold]["classification_report"][class_number][class_metric]
                                )
    for class_number in precision_dict:
        precision_mean_dict[class_number] = np.mean(precision_dict[class_number])
        precision_std_dict[class_number] = np.std(precision_dict[class_number])

    for class_number in recall_dict:
        recall_mean_dict[class_number] = np.mean(recall_dict[class_number])
        recall_std_dict[class_number] = np.std(recall_dict[class_number])

    for class_number in f1_score_dict:
        f1_score_mean_dict[class_number] = np.mean(f1_score_dict[class_number])
        f1_score_std_dict[class_number] = np.std(f1_score_dict[class_number])